# Introduction to Linear Programming with Python - Part 5
## Using PuLP with pandas and binary constraints to solve a scheduling problem

In this example, we'll be solving a scheduling problem. We have 2 offshore production plants in 2 locations and an estimated demand for our products. 

We want to produce a schedule of production from both plants that meets our demand with the lowest cost.

A factory can be in 2 states:
* Off - Producing zero units
* On - Producing between its minimum and maximum production capacities

Both factories have fixed costs, that are incurred as long as the factory is on, and variable costs, a cost per unit of production. These vary month by month. 

We also know that factory B is down for maintenance in month 5.

We'll start by importing our data. 

The data is imported into a multi-index pandas DataFrame using 'Month' and 'Factory' as our index columns.

In [174]:
import pandas as pd
import numpy as np
import itertools
import pulp

In [189]:
# Init
DAYS = [1,2,3]
HOURS = [1,2,3,4,5,6]
LESSONS = ['Math', 'Science']
CLASSES = ['A1', 'A2']
TEACHERS = ['Ruti', 'Shoshi']

lessons_columns = ['Day', 'Hour', 'Lesson', 'Class', 'Teacher']
lessons_data = pd.DataFrame(columns=lessons_columns)
lessons_cart_input = (DAYS, HOURS, LESSONS, CLASSES, TEACHERS)

teachers_const_columns = ['Teacher', 'Day', 'Hour', 'Value']
teachers_data = pd.DataFrame(columns=teachers_const_columns)
teachers_cart_input = (TEACHERS, DAYS, HOURS, [1])

edu_const_columns = ['Class', 'Lesson', 'Value']
edu_data = pd.DataFrame(columns=edu_const_columns)
edu_cart_input = (CLASSES, LESSONS, [3])

lessons_cart = itertools.product(*lessons_cart_input, repeat=1)
teachers_cart = itertools.product(*teachers_cart_input, repeat=1)
edu_cart = itertools.product(*edu_cart_input, repeat=1)

print('expected num of variables: {}'.format(len(DAYS)*len(HOURS)*len(LESSONS)*len(CLASSES)*len(TEACHERS)))

expected num of variables: 144


In [470]:
for i, out in enumerate(lessons_cart):
    lessons_data.loc[i] = out
print(lessons_data.shape)

for i, out in enumerate(teachers_cart):
    teachers_data.loc[i] = out
teachers_data = teachers_data.drop_duplicates()
print(teachers_data.shape)

for i, out in enumerate(edu_cart):
    edu_data.loc[i] = out
edu_data = edu_data.drop_duplicates()
print(edu_data.shape)

teachers_lesson_data = pd.read_csv('teacher_lessons.csv')
print(teachers_lesson_data.shape)

(144, 5)
(36, 4)
(4, 3)
(4, 3)


In [191]:
lessons_data.head(10)

,Day,Hour,Lesson,Class,Teacher
0,1,1,Math,A1,Ruti
1,1,1,Math,A1,Shoshi
2,1,1,Math,A2,Ruti
3,1,1,Math,A2,Shoshi
4,1,1,Science,A1,Ruti
5,1,1,Science,A1,Shoshi
6,1,1,Science,A2,Ruti
7,1,1,Science,A2,Shoshi
8,1,2,Math,A1,Ruti
9,1,2,Math,A1,Shoshi


In [192]:
teachers_data.head(10)

,Teacher,Day,Hour,Value
0,Ruti,1,1,1
1,Ruti,1,2,1
2,Ruti,1,3,1
3,Ruti,1,4,1
4,Ruti,1,5,1
5,Ruti,1,6,1
6,Ruti,2,1,1
7,Ruti,2,2,1
8,Ruti,2,3,1
9,Ruti,2,4,1


In [193]:
edu_data.head(10)

,Class,Lesson,Value
0,A1,Math,3
1,A1,Science,3
2,A2,Math,3
3,A2,Science,3


In [471]:
teachers_lesson_data.head(10)

,Teacher,Lesson,Value
0,Ruti,Math,1
1,Ruti,Science,0
2,Shoshi,Math,1
3,Shoshi,Science,1


In [179]:
lessons_dict = pulp.LpVariable.dicts("variables",
                                     ((row.Day, row.Hour, row.Lesson, row.Class, row.Teacher) for row in lessons_data.itertuples()),
                                     lowBound=0,
                                     cat='Binary')

We instantiate our model and use LpMinimize as the aim is to minimise costs.

## Model initialization

In [478]:
all_data = lessons_data.merge(teachers_data, on=['Teacher', 'Day', 'Hour'])

In [479]:
model = pulp.LpProblem("Minimize amount of lessons", pulp.LpMinimize)

In [480]:
# Objective function build
expr = []
for row in all_data.itertuples():
    expr += lessons_dict[row.Day, row.Hour, row.Lesson, row.Class, row.Teacher] * row.Value
model += expr

### Constraints

In [481]:
# How many hours for every class for every lesson
for edu_row in edu_data.itertuples():
    lp_sum = []
    expr = pulp.LpAffineExpression()
    for lesson_row in lessons_data[(lessons_data['Lesson'] == edu_row.Lesson) & (lessons_data['Class'] == edu_row.Class)].drop_duplicates().itertuples():
        lp_sum += lessons_dict[lesson_row.Day, lesson_row.Hour, lesson_row.Lesson, lesson_row.Class, lesson_row.Teacher]
    expr = (lp_sum == edu_row.Value)
    model += expr       
    

In [482]:
# Teachers can teach only one class per hour
for row in lessons_data[['Teacher', 'Day', 'Hour']].drop_duplicates().itertuples():
    lp_sum = []
    expr = pulp.LpAffineExpression()
    for lesson_row in lessons_data[(lessons_data['Teacher'] == row.Teacher) & (lessons_data['Day'] == row.Day) & (lessons_data['Hour'] == row.Hour)].drop_duplicates().itertuples():
        lp_sum += lessons_dict[lesson_row.Day, lesson_row.Hour, lesson_row.Lesson, lesson_row.Class, lesson_row.Teacher]
    expr = (lp_sum <= 1)
#     print(expr, '\n\n')
    model += expr   

In [483]:
# Classes can have only one lesson per hour
for row in lessons_data[['Class', 'Day', 'Hour']].drop_duplicates().itertuples():
    lp_sum = []
    expr = pulp.LpAffineExpression()
    for lesson_row in lessons_data[(lessons_data['Class'] == row.Class) & (lessons_data['Day'] == row.Day) & (lessons_data['Hour'] == row.Hour)].drop_duplicates().itertuples():
        lp_sum += lessons_dict[lesson_row.Day, lesson_row.Hour, lesson_row.Lesson, lesson_row.Class, lesson_row.Teacher]
    expr = (lp_sum <= 1)
#     print(expr, '\n\n')
    model += expr   

In [484]:
# Window variables
L_dict = pulp.LpVariable.dicts("variables",
                                     ((row.Day, row.Hour, row.Class, 'L') for row in lessons_data[['Day', 'Hour', 'Class']].drop_duplicates().itertuples()),
                                     lowBound=0,
                                     cat='Binary')

Y_dict = pulp.LpVariable.dicts("variables",
                                     ((row.Day, row.Hour, row.Class, 'Y') for row in lessons_data[['Day', 'Hour', 'Class']].drop_duplicates().itertuples()),
                                     lowBound=0,
                                     cat='Binary')

Z_dict = pulp.LpVariable.dicts("variables",
                                     ((row.Day, row.Hour, row.Class, 'Z') for row in lessons_data[['Day', 'Hour', 'Class']].drop_duplicates().itertuples()),
                                     lowBound=0,
                                     cat='Binary')

In [485]:
# Window constraints
for row, L_ijl in L_dict.items():
    expr = pulp.LpAffineExpression()
    sub_expr = pulp.LpAffineExpression()
    lp_sum = []
    for sub_row in lessons_data[(lessons_data['Day'] == row[0]) & (lessons_data['Hour'] == row[1]) & (lessons_data['Class'] == row[2])].drop_duplicates().itertuples():
        lp_sum += lessons_dict[sub_row.Day, sub_row.Hour, sub_row.Lesson, sub_row.Class, sub_row.Teacher]
    expr = (L_ijl - lp_sum == 0)
#     print(expr)
    model += expr

for row, L_ijl in L_dict.items():
    if row[1] <= max(HOURS)-1:
        expr = pulp.LpAffineExpression()
        expr = (L_dict[row] - L_dict[(row[0], row[1]+1, row[2], 'L')] + Y_dict[(row[0], row[1]+1, row[2], 'Y')] - Z_dict[(row[0], row[1]+1, row[2], 'Z')] == 0)
        model += expr

for row in lessons_data[['Class', 'Day', 'Hour']].drop_duplicates().itertuples():
    lp_sum = []
    expr = pulp.LpAffineExpression()
    for lesson_row in lessons_data[['Class', 'Day', 'Hour']][(lessons_data['Class'] == row.Class) & (lessons_data['Day'] == row.Day)].drop_duplicates().itertuples():
        lp_sum += Y_dict[lesson_row.Day, lesson_row.Hour, lesson_row.Class, 'Y']
    expr = (lp_sum <= 1)
#     print(expr, '\n\n')
    model += expr  

for row in lessons_data[['Class', 'Day', 'Hour']].drop_duplicates().itertuples():
    lp_sum = []
    expr = pulp.LpAffineExpression()
    for lesson_row in lessons_data[['Class', 'Day', 'Hour']][(lessons_data['Class'] == row.Class) & (lessons_data['Day'] == row.Day)].drop_duplicates().itertuples():
        lp_sum += Z_dict[lesson_row.Day, lesson_row.Hour, lesson_row.Class, 'Z']
    expr = (lp_sum <= 1)
#     print(expr, '\n\n')
    model += expr  
    

In [486]:
# Teacher lessons constraints
for row in lessons_data.drop_duplicates().itertuples():
    expr = pulp.LpAffineExpression()
    expr = (lessons_dict[row.Day, row.Hour, row.Lesson, row.Class, row.Teacher] <= teachers_lesson_data[(teachers_lesson_data['Teacher'] == row.Teacher) & (teachers_lesson_data['Lesson'] == row.Lesson)]['Value'])
#     print(expr, '\n\n')
    model += expr   

In [487]:
print(model)

Minimize amount of lessons:
MINIMIZE
1*variables_(1,_1,_'Math',_'A1',_'Ruti') + 1*variables_(1,_1,_'Math',_'A1',_'Shoshi') + 1*variables_(1,_1,_'Math',_'A2',_'Ruti') + 1*variables_(1,_1,_'Math',_'A2',_'Shoshi') + 1*variables_(1,_1,_'Science',_'A1',_'Ruti') + 1*variables_(1,_1,_'Science',_'A1',_'Shoshi') + 1*variables_(1,_1,_'Science',_'A2',_'Ruti') + 1*variables_(1,_1,_'Science',_'A2',_'Shoshi') + 1*variables_(1,_2,_'Math',_'A1',_'Ruti') + 1*variables_(1,_2,_'Math',_'A1',_'Shoshi') + 1*variables_(1,_2,_'Math',_'A2',_'Ruti') + 1*variables_(1,_2,_'Math',_'A2',_'Shoshi') + 1*variables_(1,_2,_'Science',_'A1',_'Ruti') + 1*variables_(1,_2,_'Science',_'A1',_'Shoshi') + 1*variables_(1,_2,_'Science',_'A2',_'Ruti') + 1*variables_(1,_2,_'Science',_'A2',_'Shoshi') + 1*variables_(1,_3,_'Math',_'A1',_'Ruti') + 1*variables_(1,_3,_'Math',_'A1',_'Shoshi') + 1*variables_(1,_3,_'Math',_'A2',_'Ruti') + 1*variables_(1,_3,_'Math',_'A2',_'Shoshi') + 1*variables_(1,_3,_'Science',_'A1',_'Ruti') + 1*variables_(

In [488]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [489]:
for k, v in lessons_dict.items():
    print(k, v.varValue)

(1, 1, 'Math', 'A1', 'Ruti') 0.0
(1, 1, 'Math', 'A1', 'Shoshi') 0.0
(1, 1, 'Math', 'A2', 'Ruti') 0.0
(1, 1, 'Math', 'A2', 'Shoshi') 0.0
(1, 1, 'Science', 'A1', 'Ruti') 0.0
(1, 1, 'Science', 'A1', 'Shoshi') 0.0
(1, 1, 'Science', 'A2', 'Ruti') 0.0
(1, 1, 'Science', 'A2', 'Shoshi') 0.0
(1, 2, 'Math', 'A1', 'Ruti') 0.0
(1, 2, 'Math', 'A1', 'Shoshi') 0.0
(1, 2, 'Math', 'A2', 'Ruti') 0.0
(1, 2, 'Math', 'A2', 'Shoshi') 0.0
(1, 2, 'Science', 'A1', 'Ruti') 0.0
(1, 2, 'Science', 'A1', 'Shoshi') 0.0
(1, 2, 'Science', 'A2', 'Ruti') 0.0
(1, 2, 'Science', 'A2', 'Shoshi') 0.0
(1, 3, 'Math', 'A1', 'Ruti') 0.0
(1, 3, 'Math', 'A1', 'Shoshi') 0.0
(1, 3, 'Math', 'A2', 'Ruti') 0.0
(1, 3, 'Math', 'A2', 'Shoshi') 0.0
(1, 3, 'Science', 'A1', 'Ruti') 0.0
(1, 3, 'Science', 'A1', 'Shoshi') 0.0
(1, 3, 'Science', 'A2', 'Ruti') 0.0
(1, 3, 'Science', 'A2', 'Shoshi') 0.0
(1, 4, 'Math', 'A1', 'Ruti') 0.0
(1, 4, 'Math', 'A1', 'Shoshi') 0.0
(1, 4, 'Math', 'A2', 'Ruti') 0.0
(1, 4, 'Math', 'A2', 'Shoshi') 0.0
(1, 4, 'Scie

# END

In [14]:
# Iterables
days = lessons_data['Day'].drop_duplicates()
hours = lessons_data['Hour'].drop_duplicates()
lessons = lessons_data['Lesson'].drop_duplicates()

In [15]:
# Each lesson has two hours
for lesson in lessons:
    expr = pulp.LpAffineExpression()
    for day in days:
        for hour in hours:
            expr += lessons_status[day, hour, lesson]
    const = (expr == 2)
    model += const

In [16]:
# No more than 6 hours in a day
for day in days:
    expr = pulp.LpAffineExpression()
    for lesson in lessons:
        for hour in hours:
            expr += lessons_status[day, hour, lesson]
    const = (expr <= 6)
    model += const

In [17]:
# See the model
model

Minimize amount of lessons:
MINIMIZE
1*lessons_(1,_1,_'Art') + 1*lessons_(1,_1,_'Bible') + 1*lessons_(1,_1,_'Education') + 1*lessons_(1,_1,_'Hebrew') + 1*lessons_(1,_1,_'Math') + 1*lessons_(1,_1,_'Music') + 1*lessons_(1,_1,_'Science') + 1*lessons_(1,_1,_'Sport') + 1*lessons_(1,_2,_'Art') + 1*lessons_(1,_2,_'Bible') + 1*lessons_(1,_2,_'Education') + 1*lessons_(1,_2,_'Hebrew') + 1*lessons_(1,_2,_'Math') + 1*lessons_(1,_2,_'Music') + 1*lessons_(1,_2,_'Science') + 1*lessons_(1,_2,_'Sport') + 1*lessons_(1,_3,_'Art') + 1*lessons_(1,_3,_'Bible') + 1*lessons_(1,_3,_'Education') + 1*lessons_(1,_3,_'Hebrew') + 1*lessons_(1,_3,_'Math') + 1*lessons_(1,_3,_'Music') + 1*lessons_(1,_3,_'Science') + 1*lessons_(1,_3,_'Sport') + 1*lessons_(1,_4,_'Art') + 1*lessons_(1,_4,_'Bible') + 1*lessons_(1,_4,_'Education') + 1*lessons_(1,_4,_'Hebrew') + 1*lessons_(1,_4,_'Math') + 1*lessons_(1,_4,_'Music') + 1*lessons_(1,_4,_'Science') + 1*lessons_(1,_4,_'Sport') + 1*lessons_(1,_5,_'Art') + 1*lessons_(1,_5,_'Bible')

In [18]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

Let's take a look at the optimal production schedule output for each month from each factory. For ease of viewing we'll output the data to a pandas DataFrame.

In [19]:
for row in lessons_data.itertuples():
    print(lessons_status[row.Day, row.Hour, row.Lesson],\
          lessons_status[row.Day, row.Hour, row.Lesson].varValue)

lessons_(1,_1,_'Math') 0.0
lessons_(1,_1,_'Science') 0.0
lessons_(1,_1,_'Hebrew') 1.0
lessons_(1,_1,_'Bible') 0.0
lessons_(1,_1,_'Music') 0.0
lessons_(1,_1,_'Education') 0.0
lessons_(1,_1,_'Sport') 0.0
lessons_(1,_1,_'Art') 0.0
lessons_(1,_2,_'Math') 0.0
lessons_(1,_2,_'Science') 0.0
lessons_(1,_2,_'Hebrew') 1.0
lessons_(1,_2,_'Bible') 0.0
lessons_(1,_2,_'Music') 0.0
lessons_(1,_2,_'Education') 0.0
lessons_(1,_2,_'Sport') 0.0
lessons_(1,_2,_'Art') 1.0
lessons_(1,_3,_'Math') 0.0
lessons_(1,_3,_'Science') 0.0
lessons_(1,_3,_'Hebrew') 0.0
lessons_(1,_3,_'Bible') 0.0
lessons_(1,_3,_'Music') 0.0
lessons_(1,_3,_'Education') 0.0
lessons_(1,_3,_'Sport') 0.0
lessons_(1,_3,_'Art') 0.0
lessons_(1,_4,_'Math') 0.0
lessons_(1,_4,_'Science') 0.0
lessons_(1,_4,_'Hebrew') 0.0
lessons_(1,_4,_'Bible') 0.0
lessons_(1,_4,_'Music') 0.0
lessons_(1,_4,_'Education') 0.0
lessons_(1,_4,_'Sport') 0.0
lessons_(1,_4,_'Art') 0.0
lessons_(1,_5,_'Math') 0.0
lessons_(1,_5,_'Science') 0.0
lessons_(1,_5,_'Hebrew') 0.0
le

In [14]:
output = []
for month, factory in production:
    var_output = {
        'Month': month,
        'Factory': factory,
        'Production': production[(month, factory)].varValue,
        'Factory Status': factory_status[(month, factory)].varValue
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Month', 'Factory'])
output_df.set_index(['Month', 'Factory'], inplace=True)
output_df

Factory Status  Production
Month Factory                            
1     A                   1.0     70000.0
      B                   1.0     50000.0
2     A                   1.0     45000.0
      B                   1.0     55000.0
3     A                   1.0     70000.0
      B                   1.0     60000.0
4     A                   1.0     30000.0
      B                   1.0    100000.0
5     A                   1.0    140000.0
      B                   0.0         0.0
6     A                   1.0     60000.0
      B                   1.0     70000.0
7     A                   1.0     90000.0
      B                   1.0     60000.0
8     A                   1.0     70000.0
      B                   1.0    100000.0
9     A                   1.0    100000.0
      B                   1.0    100000.0
10    A                   1.0    190000.0
      B                   0.0         0.0
11    A                   1.0     80000.0
      B                   1.0     60000.0
12    A                   1.0    100000.0
      B                   0.0         0.0

Notice above that the factory status is 0 when not producing and 1 when it is producing

In [77]:
# Print our objective function value (Total Costs)
print (pulp.value(model.objective))

0.0
